In [ ]:
3!pip install tensorflow-gpu

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf

In [ ]:
import numpy as np
import os
from tensorflow.keras.models import load_model, Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, BatchNormalization, Dropout, Flatten, Activation
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, LearningRateScheduler, ModelCheckpoint
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
train_path ='/content/drive/MyDrive/Seventh Semester Project/Datasets/Train'
valid_path = '/content/drive/MyDrive/Seventh Semester Project/Datasets/Validate'
test_path = '/content/drive/MyDrive/Seventh Semester Project/Datasets/Test'
Img_size = [224, 224]
batch_size = 512

In [ ]:
## Data Augmentation

train_generator =ImageDataGenerator(rescale = 1/255,
                                   horizontal_flip=True,
                                   vertical_flip=True,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   rotation_range=30,
                                   fill_mode='nearest')
valid_generator = ImageDataGenerator(rescale= 1/255)
test_generator = ImageDataGenerator(rescale = 1/ 255)

In [ ]:
# Dataset 

train_datset = train_generator.flow_from_directory(directory=train_path,
                                                   target_size= Img_size,
                                                   class_mode = 'sparse',
                                                   batch_size = batch_size,
                                                   color_mode = 'rgb', )

valid_datset = train_generator.flow_from_directory(directory=valid_path,
                                                   target_size= Img_size,
                                                   class_mode = 'sparse',
                                                   batch_size = batch_size,
                                                   color_mode = 'rgb', )

test_datset = train_generator.flow_from_directory(directory=test_path,
                                                   target_size= Img_size,
                                                   class_mode = 'sparse',
                                                   batch_size = batch_size,
                                                   color_mode = 'rgb', )

Found 10081 images belonging to 10 classes.
Found 2348 images belonging to 10 classes.
Found 2029 images belonging to 10 classes.


In [ ]:
from keras.applications.inception_v3 import InceptionV3,preprocess_input,decode_predictions

In [ ]:
base_model  = InceptionV3(weights = 'imagenet', include_top=False, input_shape=Img_size +[3])
print('loaded model')

loaded model


In [ ]:
for layer in base_model.layers:
  layer.trainable = False

In [ ]:
benchmark=Sequential()
benchmark.add(base_model)
#model.add(Dropout(0.5))
benchmark.add(Flatten())
benchmark.add(Dense(10,activation='softmax'))

In [ ]:
benchmark.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 inception_v3 (Functional)   (None, 5, 5, 2048)        21802784  
                                                                 
 flatten (Flatten)           (None, 51200)             0         
                                                                 
 dense (Dense)               (None, 225)               11520225  
                                                                 
Total params: 33,323,009
Trainable params: 11,520,225
Non-trainable params: 21,802,784
_________________________________________________________________


In [ ]:
opt=tf.keras.optimizers.Adam(lr=0.001)

benchmark.compile(loss='sparse_categorical_crossentropy',
              metrics=['accuracy'],
              optimizer=opt)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [ ]:
# Here Early stopping callback is added for unnecessary running of epochs if model performance is poor 
filepath = '/content/drive/MyDrive/Seventh Semester Project/model/InceptionV3_base_model.h5'

earlystopping = EarlyStopping(monitor = 'val_accuracy', 
                              mode = 'max' , 
                              patience = 10,
                              verbose = 1)

# for saving best model till execution
checkpoint    = ModelCheckpoint(filepath, 
                                monitor = 'val_accuracy', 
                                mode='max', 
                                save_best_only=True, 
                                verbose = 1)

callback_list = [earlystopping, checkpoint]

In [ ]:
result = benchmark.fit(train_datset, 
                   validation_data=valid_datset, 
                   epochs = 100,
                   callbacks = callback_list)
                    #verbose= 1)

Epoch 1/100
20/20 [==============================] - ETA: 0s - loss: 12.5142 - accuracy: 0.2431  
Epoch 00001: val_accuracy improved from -inf to 0.32794, saving model to /content/drive/MyDrive/Seventh Semester Project/model/InceptionV3_base_model.h5
20/20 [==============================] - 2176s 109s/step - loss: 12.5142 - accuracy: 0.2431 - val_loss: 7.4803 - val_accuracy: 0.3279
Epoch 2/100
20/20 [==============================] - ETA: 0s - loss: 3.9511 - accuracy: 0.5066 
Epoch 00002: val_accuracy improved from 0.32794 to 0.45698, saving model to /content/drive/MyDrive/Seventh Semester Project/model/InceptionV3_base_model.h5
20/20 [==============================] - 319s 16s/step - loss: 3.9511 - accuracy: 0.5066 - val_loss: 3.2158 - val_accuracy: 0.4570
Epoch 3/100
20/20 [==============================] - ETA: 0s - loss: 2.0002 - accuracy: 0.5926 
Epoch 00003: val_accuracy improved from 0.45698 to 0.47019, saving model to /content/drive/MyDrive/Seventh Semester Project/model/Incept